### <a name='top'> OSRS Bot Detection</a>
----  
Author: [Tyler Blair](https://github.com/tblair7)

__Go to:__  
<a href=#scrape_names>Scrape names</a>  
<a href=#scrape_skills>Scrape skills of all names</a>  
<a href=#skillstable>Skills table</a>  
<a href=#old>Deprecated work</a>

<a href=#top>Back to top</a>  
### <a name='scrape_names'> Scrape names from hiscores </a>  

In [195]:
def scrape_names(min_rank, max_rank, skill):
    import requests
    import sqlite3 as sql
    from datetime import datetime
    from bs4 import BeautifulSoup
    import hs_tables
    
    conn = sql.connect('data/osrs.db')
    
    # create players table if it doesn't already exist
    # attributes: name, date, skill
    hs_tables.create_players_table(conn)
    
    # get skill name for inserting attribute in datebase table
    skills = ["Overall", "Attack", "Defence", "Strength", "Hitpoints", "Ranged", "Prayer", "Magic", "Cooking",
              "Woodcutting", "Fletching", "Fishing", "Firemaking", "Crafting", "Smithing", "Mining", "Herblore",
              "Agility", "Thieving", "Slayer", "Farming", "Runecraft", "Hunter", "Construction"]
    skill_name = skills[skill]
    
    # time in utc for later time series analysis
    now = datetime.utcnow()
    time = '{0}-{1:02d}-{2:02d} {3:02d}:{4:02d}:{5:02d}'.format(now.year, now.month, now.day, 
                                                                now.hour, now.minute, now.second)
    
    total_base = 'https://secure.runescape.com/m=hiscore_oldschool/overall.ws?table={0}&page='.format(skill)
    #player_url_base = 'https://secure.runescape.com/m=hiscore_oldschool/index_lite.ws?player='
    
    # range of pages to scrape results
    min_page = min_rank//25
    max_page = max_rank//25
    pages = max_page - min_page
    #url_no_page = ranking_url_base + 'table={22}&page='.format(table_num)
    
    for i in range(min_page, max_page):
        
        url = total_base + str(i)
        soup = BeautifulSoup(requests.get(url).text)
        try:
            table = soup.find("tbody")
        # create catch here
        except:
            return soup
        
        for row in table.findAll("tr"):
            player = row.findAll("td")
            # cols = [rank, name, level, xp]
            cols = [element.text.strip() for element in player]
            #name = cols[1].replace(u'\xa0', u' ')
            
            if cols[1]:
                name = cols[1].replace(u'\xa0', u' ')
                #print(name)
                rank = int(cols[0].replace(u',', u''))
                try:
                    conn.execute('''INSERT OR IGNORE INTO players(name, date, skill, rank) VALUES (?,?,?,?)''',(name, time, skill_name, int(rank)))
                    conn.commit()
                except:
                    print('Failed:', cols)
                    continue
            
            else:
                continue
            
        if not i%20:
            print('Pages scraped: {0}/{1}'.format(i-min_page, pages), datetime.utcnow() - now)
        else:
            continue
            
    return table

In [7]:
# scrape hiscores for names (construction ranks 440000 to 561000, level 50 con)
try:
    import importlib
    importlib.reload(scrape_hs)
    print('Reimported scrape_hs.py')
except:
    import scrape_hs

# 1400 end, 35000 done
min_rank = 485000 #440000 min originally
max_rank = 561000
skill = 23 # construction = 23

table = scrape_hs.scrape_names(min_rank, max_rank, skill)

Pages scraped: 0/3040 2019-05-23 00:42:32.248702
Pages scraped: 10/3040 2019-05-23 00:42:52.417858
Pages scraped: 90/3040 2019-05-23 00:47:09.248114
Pages scraped: 110/3040 2019-05-23 00:48:39.260316
Pages scraped: 130/3040 2019-05-23 00:50:06.607790
Pages scraped: 150/3040 2019-05-23 00:51:33.753334
Pages scraped: 160/3040 2019-05-23 00:52:19.528423
Pages scraped: 180/3040 2019-05-23 00:53:51.589447
Pages scraped: 250/3040 2019-05-23 00:58:54.393519
Pages scraped: 280/3040 2019-05-23 01:01:07.680285
Pages scraped: 300/3040 2019-05-23 01:02:33.434284
Pages scraped: 310/3040 2019-05-23 01:03:18.902874
Pages scraped: 430/3040 2019-05-23 01:12:08.331737
Pages scraped: 440/3040 2019-05-23 01:12:48.781933
Pages scraped: 450/3040 2019-05-23 01:13:35.068626
Pages scraped: 470/3040 2019-05-23 01:15:06.100727
Pages scraped: 480/3040 2019-05-23 01:15:51.974203
Pages scraped: 500/3040 2019-05-23 01:17:18.609096
Pages scraped: 600/3040 2019-05-23 01:24:39.643897
Pages scraped: 620/3040 2019-05-23 

<a href=#top>Back to top</a>  
### <a name='scrape_skills'> Scrape skills of each player </a>  

In [2]:
import requests
import pandas
import pyspark
import sqlite3 as sql
import pandas as pd
import pickle as pckl
from bs4 import BeautifulSoup
from pyspark.sql.types import DateType
from pyspark.sql import SparkSession
from datetime import datetime

conn = sql.connect('data/osrs.db')

spark = SparkSession \
    .builder \
    .appName("osrs")\
    .getOrCreate()

sqlContext = pyspark.sql.SQLContext(spark)#sc = pyspark.SparkContext

In [14]:
def test(players):
    return (players, 

In [15]:
conn = sql.connect('data/osrs.db')
names = conn.execute('''SELECT name FROM players_name LIMIT 30''').fetchall()
df = pd.DataFrame({'name': names})
sdf = sqlContext.createDataFrame(df)
sdf2 = sdf.select('name').repartition(4)

a = sdf2.rdd.mapPartitions(test).collect()
a

[Row(name=Row(_1='Supreme_v1')),
 Row(name=Row(_1='M3EK MlLLY')),
 Row(name=Row(_1='MrTrunks')),
 Row(name=Row(_1='OBG Nicola')),
 Row(name=Row(_1='redromper')),
 Row(name=Row(_1='SSGvegito')),
 Row(name=Row(_1='YES vote')),
 Row(name=Row(_1='TMGGuthan')),
 Row(name=Row(_1='Kyle SR')),
 Row(name=Row(_1='Von Disney')),
 Row(name=Row(_1='thafamilia3')),
 Row(name=Row(_1='buga shuga')),
 Row(name=Row(_1='sweetslaps')),
 Row(name=Row(_1='lceef')),
 Row(name=Row(_1='La Tigers 44')),
 Row(name=Row(_1='Zephozzz')),
 Row(name=Row(_1='Grant2k1')),
 Row(name=Row(_1='CleanedTotal')),
 Row(name=Row(_1='Space Mankey')),
 Row(name=Row(_1='Rongdre')),
 Row(name=Row(_1='Stab Em All2')),
 Row(name=Row(_1='Hellz Zerker')),
 Row(name=Row(_1='ChrisW247')),
 Row(name=Row(_1='Logless')),
 Row(name=Row(_1='M d m a zing')),
 Row(name=Row(_1='PhantomsCore')),
 Row(name=Row(_1='young pimpin')),
 Row(name=Row(_1='Maximus Zerk')),
 Row(name=Row(_1='L55')),
 Row(name=Row(_1='Badger64'))]

In [35]:
def get_single_player_part(players):
    from datetime import datetime
    # base url for api request
    player_url_base = 'https://secure.runescape.com/m=hiscore_oldschool/index_lite.ws?player='

    i = 0
    now = datetime.utcnow()
    
    for player in players:
        name = player['name']
        url = player_url_base + name
        
        if not i%100:
            now = datetime.utcnow()
        else:
            pass
        i += 1
        
        try:
            page = requests.get(url).text.replace(u'\n', u' ')
            skills = [i.split(',') for i in page.split()]
            xp = [i[2] for i in skills[0:24]]
            rank = [i[0] for i in skills[0:24]]
            yield tuple((name, now, xp, rank))# players_data
        except:
            continue
    return

In [18]:
players = conn.execute('SELECT name, date FROM players_name LIMIT 50').fetchall()
players[0]

('Hellz Zerker', '2019-05-22 19:34:35')

In [40]:
players = conn.execute('SELECT name, date FROM players_name').fetchall()
players_df = pd.DataFrame({'name': [x[0] for x in players], 'date': [x[1] for x in players]})
players_sdf = sqlContext.createDataFrame(players_df)

In [41]:
players = players_sdf.select('name').repartition(16)
players_hs = players.rdd.mapPartitions(get_single_player_part)
players_hs.is_checkpointed = True
print('Checkpoint enabled: ', players_hs.is_checkpointed,'\nNum partitions: ', players_hs.getNumPartitions())

Checkpoint enabled:  True 
Num partitions:  16


In [45]:
print(datetime.utcnow())
players_results2 = players_hs.collect()
print(datetime.utcnow())
players_results2[0]

2019-05-30 03:17:51.603598
2019-05-31 07:54:07.501399


('Ms Jinx',
 datetime.datetime(2019, 5, 30, 3, 17, 54, 48099),
 ['11968400',
  '1122847',
  '757863',
  '738270',
  '1167841',
  '785925',
  '366463',
  '729736',
  '1121586',
  '641689',
  '297706',
  '572190',
  '192952',
  '741736',
  '345522',
  '274134',
  '244771',
  '184051',
  '308406',
  '574393',
  '399890',
  '106458',
  '184845',
  '109126'],
 ['443845',
  '614330',
  '638305',
  '972125',
  '797633',
  '907926',
  '503780',
  '826204',
  '371592',
  '558650',
  '666263',
  '527408',
  '709161',
  '361890',
  '432763',
  '691228',
  '384239',
  '717141',
  '303674',
  '522748',
  '337210',
  '330905',
  '485324',
  '459787'])

In [48]:
pckl.dump(players_results3, open('data/players_results0601.pckl', 'wb'))

In [47]:
print(datetime.utcnow())
players_results3 = players_hs.collect()
print(datetime.utcnow())
players_results3[0]

2019-05-31 07:54:08.876105
2019-06-01 12:44:02.238793


('Ms Jinx',
 datetime.datetime(2019, 5, 31, 7, 54, 11, 100306),
 ['11968400',
  '1122847',
  '757863',
  '738270',
  '1167841',
  '785925',
  '366463',
  '729736',
  '1121586',
  '641689',
  '297706',
  '572190',
  '192952',
  '741736',
  '345522',
  '274134',
  '244771',
  '184051',
  '308406',
  '574393',
  '399890',
  '106458',
  '184845',
  '109126'],
 ['444319',
  '614864',
  '638913',
  '972847',
  '798341',
  '908676',
  '504331',
  '827160',
  '372013',
  '559032',
  '666833',
  '527872',
  '709905',
  '362367',
  '433385',
  '691906',
  '384682',
  '717868',
  '304011',
  '523313',
  '337749',
  '331297',
  '485842',
  '460365'])

In [34]:
import importlib
try:
    importlib.reload(scrape_hs)
    print('Reimported scrape_hs')
except:
    import scrape_hs

xp_df, rank_df = scrape_hs.get_all_players()

Reimported scrape_hs
Checkpoint enabled:  True 
Num partitions:  16
2019-05-28 21:04:03.673500


ValueError: not enough values to unpack (expected 2, got 0)

In [28]:
import pickle as pckl
#results = pckl.load(open('data/players_results1.pckl', 'rb'))

In [33]:
a = results[0:2].copy()
for row in a:
    xp = [row[0]] + [row[1]] + [int(x) for x in row[2]]
    rank = [row[0]] + [row[1]] + [int(x) for x in row[3]]
    
xp

['Z A T 0 XX',
 datetime.datetime(2019, 5, 24, 0, 54, 9, 51961),
 14392616,
 309851,
 65507,
 2816878,
 1667719,
 1977078,
 125434,
 1484428,
 783341,
 201439,
 15791,
 166138,
 116305,
 318242,
 182060,
 286286,
 62756,
 215569,
 305446,
 168996,
 46663,
 21961,
 2949725,
 105003]

In [2]:
#df = spark.read.load('data/player_names.parquet').repartition(16)

In [12]:
#pckl.dump(players_results, open('data/players_results528.pckl', 'wb'))

In [8]:
players_xp = pckl.load(open('data/players_xp.pckl', 'rb'))
players_xp.head()

,name,date,overall,attack,defence,strength,hitpoints,ranged,prayer,magic,...,smithing,mining,herblore,agility,thieving,slayer,farming,runecraft,hunter,construction
0,Mjpel3,2019-05-24 00:54:09.051961,16294050,1031276,860741,1528457,2219614,3259345,744358,1458143,...,104648,749672,79803,524779,169396,498128,274860,120365,125401,102478
1,Z A T 0 XX,2019-05-24 00:54:09.051961,14392616,309851,65507,2816878,1667719,1977078,125434,1484428,...,182060,286286,62756,215569,305446,168996,46663,21961,2949725,105003
2,Elliebus23,2019-05-24 00:54:09.051961,48627220,2476953,2054502,2026809,3650780,2473785,892909,2586039,...,131354,480326,88082,796701,154521,1902393,262975,81436,1999849,106951
3,SloppySlay,2019-05-24 00:54:09.051961,13665012,2084263,1557064,1441797,2289488,1759085,1989926,1261546,...,12743,22329,4045,29972,97299,485469,0,16932,3129,102327
4,Dylan Is Bad,2019-05-24 00:54:09.051961,33228886,3351083,2101649,5381825,5410943,4224400,759214,5746639,...,46939,302305,29436,826671,168041,2125762,37583,51158,273837,102872


In [16]:
short_players = players_results[0:100].copy()

In [7]:
columns = ["name", "date", "Overall", "Attack", "Defence", "Strength", "Hitpoints", "Ranged", "Prayer", "Magic", "Cooking",
              "Woodcutting", "Fletching", "Fishing", "Firemaking", "Crafting", "Smithing", "Mining", "Herblore",
              "Agility", "Thieving", "Slayer", "Farming", "Runecraft", "Hunter", "Construction"]
columns = [x.lower() for x in columns]

In [66]:
results = players_results + players_results2
len(results)

140379

In [89]:
def data_to_df(data, df, table):
    
    columns = ['name', 'date', 'overall', 'attack', 'defence', 'strength', 'hitpoints', 'ranged', 'prayer', 'magic', 
               'cooking', 'woodcutting', 'fletching', 'fishing', 'firemaking', 'crafting', 'smithing', 'mining', 
               'herblore', 'agility', 'thieving', 'slayer', 'farming', 'runecraft', 'hunter', 'construction']
    
    # which data to parse, rank = 3rd entry in tuple, xp = 2nd entry in tuple
    if table == 'rank':
        keep = 3
    else:
        keep = 2
        
    data_list = []
        
    for row in data:
        insert = [row[0]] + [row[1]] + [int(x) for x in row[keep]]
        key_value = zip(columns, insert)
        
        my_dict = {}
    
        for key, value in key_value:
            my_dict[key] = value
            
        data_list.append(my_dict)
    
    df = df.append(data_list)
        
    return df

In [97]:
two_days = data_to_df(results, players_xp, 'xp')

In [98]:
#pckl.dump(two_days, open('data/players_xp.pckl', 'wb'))

In [9]:
xp_df = sqlContext.createDataFrame(players_xp)

In [10]:
xp_df.take(5)

[Row(name='Mjpel3', date=datetime.datetime(2019, 5, 24, 0, 54, 9, 51961), overall=16294050, attack=1031276, defence=860741, strength=1528457, hitpoints=2219614, ranged=3259345, prayer=744358, magic=1458143, cooking=769577, woodcutting=588335, fletching=127612, fishing=676967, firemaking=106485, crafting=173610, smithing=104648, mining=749672, herblore=79803, agility=524779, thieving=169396, slayer=498128, farming=274860, runecraft=120365, hunter=125401, construction=102478),
 Row(name='Z A T 0 XX', date=datetime.datetime(2019, 5, 24, 0, 54, 9, 51961), overall=14392616, attack=309851, defence=65507, strength=2816878, hitpoints=1667719, ranged=1977078, prayer=125434, magic=1484428, cooking=783341, woodcutting=201439, fletching=15791, fishing=166138, firemaking=116305, crafting=318242, smithing=182060, mining=286286, herblore=62756, agility=215569, thieving=305446, slayer=168996, farming=46663, runecraft=21961, hunter=2949725, construction=105003),
 Row(name='Elliebus23', date=datetime.da

<a name=#skillstable></a>  
<a href=#top>Back to top</a>  
  
__Tables (Skills)__  
  
0 - Overall  
1 - Attack  
2 - Defence  
3 - Strength  
4 - Hitpoints  
5 - Ranged  
6 - Prayer  
7 - Magic  
8 - Cooking  
9 - Woodcutting  
10 - Fletching  
11 - Fishing  
12 - Firemaking  
13 - Crafting  
14 - Smithing  
15 - Mining  
16 - Herblore  
17 - Agility  
18 - Thieving  
19 - Slayer  
20 - Farming  
21 - Runecraft  
22 - Hunter  
23 - Construction  

<a href=#top>Back to top</a>  
### <a name='old'> Deprecated worked, since refactored </a>  

In [28]:
name = 'Tyler'
xp = 100
skill = 'att'
tuple((name, xp, skill))

('Tyler', 100, 'att')

In [8]:
players = df.select('name').limit(5)
players.collect()

[Row(name='IDK Hunter'),
 Row(name='Evil rake'),
 Row(name='Hoibur'),
 Row(name='madnesh'),
 Row(name='AGM-86')]

In [42]:
total = 0
for i in range(len(players_results)):
    total += len(players_results[i])
total

20

In [8]:
def get_single_player(players):
    from datetime import datetime
    #print(player['name'])
    # base url for api request
    player_url_base = 'https://secure.runescape.com/m=hiscore_oldschool/index_lite.ws?player='
    
    for player in players:
        name = player['name']
        url = player_url_base + name
    
        now = datetime.utcnow()
        time = '{0}-{1:02d}-{2:02d} {3:02d}:{4:02d}:{5:02d}'.format(now.year, now.month, now.day, now.hour, now.minute, now.second)
        
        try:
            page = requests.get(url).text.replace(u'\n', u' ')
            skills = [i.split(',') for i in page.split()]
            xp = [i[2] for i in skills[0:24]]
            rank = [i[0] for i in skills[0:24]]
        except:
            print(name)
            continue
    return (name, time, xp, rank)

In [ ]:
def get_player(conn, skill):
    # base url for api request
    player_url_base = 'https://secure.runescape.com/m=hiscore_oldschool/index_lite.ws?player='
    
    skills = ["Overall", "Attack", "Defence", "Strength", "Hitpoints", "Ranged", "Prayer", "Magic", "Cooking",
              "Woodcutting", "Fletching", "Fishing", "Firemaking", "Crafting", "Smithing", "Mining", "Herblore",
              "Agility", "Thieving", "Slayer", "Farming", "Runecraft", "Hunter", "Construction"]
    skill_name = skills[skill]
    print(skill_name)
    
    players = conn.execute('''SELECT name from players_name WHERE skill = (?)''', (skill_name,)).fetchall()
    num_players = len(players)
    
    now = datetime.utcnow()
    time = '{0}-{1:02d}-{2:02d} {3:02d}:{4:02d}:{5:02d}'.format(now.year, now.month, now.day, 
                                                                now.hour, now.minute, now.second)
    failed = []
    num = 0
    
    for player in players:
        # sql returns list of tuples, this isolates the name
        name = player[0]
        url = player_url_base + name
        
        num += 1
        if not num%20:
            print('Players scraped: {0}/{1}'.format(i, num_players), datetime.utcnow() - now)
        else:
            pass
        
        try:
            page = requests.get(url).text.replace(u'\n', u' ')
            skills = [i.split(',') for i in page.split()]
            xp = [i[2] for i in skills[0:24]]
            rank = [i[0] for i in skills[0:24]]
        except:
            print(name)
            failed.append(name)
            continue
            
        conn.execute('''INSERT INTO players_xp VALUES(?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)''', [name, time] + xp)
        conn.execute('''INSERT INTO players_rank VALUES(?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)''', [name, time] + rank)
            
    return failed

In [3]:
# get spark df schema
df.schema

StructType(List(StructField(name,StringType,true),StructField(date,DateType,true),StructField(skill,StringType,true),StructField(rank,LongType,true)))

In [35]:
# cast date type, no longer using this
from pyspark.sql.types import DateType
df = df.withColumn('date', df['date'].cast(DateType()))

In [42]:
df.write.mode('overwrite').parquet('data/player_names.parquet')

Py4JJavaError: An error occurred while calling o166.parquet.
: org.apache.spark.SparkException: Job aborted.
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.write(FileFormatWriter.scala:198)
	at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelationCommand.run(InsertIntoHadoopFsRelationCommand.scala:159)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult$lzycompute(commands.scala:104)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult(commands.scala:102)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.doExecute(commands.scala:122)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$execute$1.apply(SparkPlan.scala:131)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$execute$1.apply(SparkPlan.scala:127)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$executeQuery$1.apply(SparkPlan.scala:155)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope

Get player names and insert into osrs.db

In [122]:
def create_players_table(conn):
    
    conn.execute('''CREATE TABLE IF NOT EXISTS players(
                    "name" TEXT PRIMARY KEY,
                    "date" DATETIME NOT NULL,
                    "skill" TEXT,
                    "rank" INTEGER)''')
    conn.commit()

In [25]:
a = conn.execute('''SELECT name from players_name''').fetchall()
len(a)

47365

In [4]:
conn.execute('''CREATE TABLE IF NOT EXISTS players_name(
                    "name" TEXT, 
                    "date" DATETIME NOT NULL, 
                    "skill" TEXT, 
                    "rank" INTEGER)''')

1


In [5]:
conn.execute('''PRAGMA table_info("players_name")''').fetchall()

[(0, 'name', 'TEXT', 0, None, 0),
 (1, 'date', 'DATETIME', 1, None, 0),
 (2, 'skill', 'TEXT', 0, None, 0),
 (3, 'rank', 'INTEGER', 0, None, 0)]

In [17]:
df = df.append({'name': 'Tyler', 'date': 'now', 'skill': 'abc', 'rank': 100}, ignore_index=True)
df

,name,date,skill,rank
0,Tyler,now,abc,100.0
1,Tyler,now,abc,100.0


In [182]:
conn.execute('''SELECT * from players''').fetchall()

[('Hellz Zerker', '2019-05-22 19:34:35', 'Construction', 439976),
 ('Grant2k1', '2019-05-22 19:34:35', 'Construction', 439977),
 ('Kyle SR', '2019-05-22 19:34:35', 'Construction', 439978),
 ('Supreme_v1', '2019-05-22 19:34:35', 'Construction', 439979),
 ('M3EK MlLLY', '2019-05-22 19:34:35', 'Construction', 439980),
 ('Von Disney', '2019-05-22 19:34:35', 'Construction', 439981),
 ('ChrisW247', '2019-05-22 19:34:35', 'Construction', 439982),
 ('CleanedTotal', '2019-05-22 19:34:35', 'Construction', 439983),
 ('buga shuga', '2019-05-22 19:34:35', 'Construction', 439984),
 ('OBG Nicola', '2019-05-22 19:34:35', 'Construction', 439985),
 ('M d m a zing', '2019-05-22 19:34:35', 'Construction', 439986),
 ('MrTrunks', '2019-05-22 19:34:35', 'Construction', 439987),
 ('Logless', '2019-05-22 19:34:35', 'Construction', 439988),
 ('thafamilia3', '2019-05-22 19:34:35', 'Construction', 439989),
 ('SSGvegito', '2019-05-22 19:34:35', 'Construction', 439990),
 ('Space Mankey', '2019-05-22 19:34:35', 'Con

In [179]:
conn.close()
conn = sql.connect('data/osrs.db')

conn.execute('''DROP TABLE players''')

In [3]:
import time
int(time.time())

1559069497

In [43]:
def create_hs_tables(conn):
    
    conn.execute('''CREATE TABLE IF NOT EXISTS player_xp (
                    name TEXT PRIMARY KEY,
                    date DATETIME NOT NULL,
                    overall INTEGER,
                    attack INTEGER,
                    defence INTEGER,
                    strength INTEGER,
                    hitpoints INTEGER,
                    ranged INTEGER,
                    prayer INTEGER,
                    magic INTEGER,
                    cooking INTEGER,
                    woodcutting INTEGER,
                    fletching INTEGER,
                    fishing INTEGER,
                    firemaking INTEGER,
                    crafting INTEGER,
                    smithing INTEGER,
                    mining INTEGER,
                    herblore INTEGER,
                    agility INTEGER,
                    thieving INTEGER,
                    slayer INTEGER,
                    farming INTEGER,
                    runecraft INTEGER,
                    hunter INTEGER,
                    construction INTEGER)''')

    conn.execute('''CREATE TABLE IF NOT EXISTS player_rank (
                    name TEXT PRIMARY KEY,
                    date DATETIME NOT NULL,
                    overall INTEGER,
                    attack INTEGER,
                    defence INTEGER,
                    strength INTEGER,
                    hitpoints INTEGER,
                    ranged INTEGER,
                    prayer INTEGER,
                    magic INTEGER,
                    cooking INTEGER,
                    woodcutting INTEGER,
                    fletching INTEGER,
                    fishing INTEGER,
                    firemaking INTEGER,
                    crafting INTEGER,
                    smithing INTEGER,
                    mining INTEGER,
                    herblore INTEGER,
                    agility INTEGER,
                    thieving INTEGER,
                    slayer INTEGER,
                    farming INTEGER,
                    runecraft INTEGER,
                    hunter INTEGER,
                    construction INTEGER)''')

(['MattsMauler',
  'Zen Santa',
  'Dr McGroober',
  'Bird Brained',
  'BoxOfCondums',
  'Brother Boof',
  'Typulsion',
  'Boyo Bob',
  'High Pro',
  'Emperator',
  'Mabaza',
  'Tokkurainen',
  'Akaaay',
  'leathalchick',
  'KinG CarrotI',
  'Truitedulac3',
  'Lightworks',
  'Lukeed113',
  '100 hp',
  'Sushi Phil',
  'HappinessLTD',
  'RevToo',
  'zdog520',
  'Embezzlez',
  'ICESKATE'],
 ['440,000', 'ICESKATE', '51', '112,073'])

In [75]:
# not truncating time anymore
time = '{0}-{1:02d}-{2:02d} {3:02d}:{4:02d}:{5:02d}'.format(now.year, now.month, now.day, now.hour, now.minute, now.second)
time

'2019-05-22 18:05:33'

In [64]:
# table_num from skills list above, e.g., table 9 = woodcutting
def scrape_rank(table_num, num_players):
    
    conn = sql.connect('osrs_hs.db')
    
    max_pages = num_players//25
    url_no_page = ranking_url_base + 'table={0}&page='.format(table_num)
    
    for i in range(max_pages):
        url = url_no_page + str(i)
        soup = BeautifulSoup(requests.get(url).text)
        try:
            table = soup.find("tbody")
        except:
            return players
        
        for row in table.findAll("tr"):
            player = row.findAll("td")
            cols = [element.text.strip() for element in player]
            players.append(cols[1])
            
    return players

In [4]:
# define base URLs for later queries
# ranking URL ends in: table={0}&page={1} where table number is in order of skills list
ranking_url_base = 'https://secure.runescape.com/m=hiscore_oldschool/overall.ws?' 
player_url_base = 'https://secure.runescape.com/m=hiscore_oldschool/index_lite.ws?player='